In [3]:
import os
import time
import csv
import numpy as np

def calcular_diversidade_maxima_rapida(matriz_distancias, k, p):
    n = len(matriz_distancias)
    melhor_diversidade = 0
    melhor_selecao = []

    # Calcular a soma das distâncias de cada vértice
    soma_distancias = np.sum(matriz_distancias, axis=1)

    # Selecionar os 'p' vértices com maior soma de distâncias
    vertices_iniciais = np.argsort(-soma_distancias)[:p]
    
    for i in vertices_iniciais:
        selecao_atual = [i]
        diversidade_atual = 0
        candidatos = np.arange(n)
        candidatos = np.delete(candidatos, i)
        
        # Ganhos iniciais para todos os candidatos
        ganhos = matriz_distancias[candidatos, i]
        
        while len(selecao_atual) < k:
            # Selecionar o vértice com maior ganho
            melhor_idx = np.argmax(ganhos)
            melhor_adicao = candidatos[melhor_idx]
            melhor_ganho = ganhos[melhor_idx]
            
            selecao_atual.append(melhor_adicao)
            diversidade_atual += melhor_ganho
            
            # Atualizar candidatos e ganhos
            candidatos = np.delete(candidatos, melhor_idx)
            if len(candidatos) > 0:
                ganhos = np.delete(ganhos, melhor_idx)
                ganhos += matriz_distancias[candidatos, melhor_adicao]
        
        if diversidade_atual > melhor_diversidade:
            melhor_diversidade = diversidade_atual
            melhor_selecao = selecao_atual

    return melhor_diversidade, melhor_selecao


def ler_instancia_txt(caminho_arquivo):
    with open(caminho_arquivo, 'r') as file:
        n, k = map(int, file.readline().strip().split())
        matriz_distancias = np.zeros((n, n))
        for linha in file:
            v1, v2, peso = linha.strip().split()
            v1, v2, peso = int(v1), int(v2), float(peso)
            matriz_distancias[v1][v2] = peso
            matriz_distancias[v2][v1] = peso
    return matriz_distancias, k


def solver_diversidade(instancia_file):
    matriz_distancias, k = ler_instancia_txt(instancia_file)
    n = len(matriz_distancias)

    start_time = time.time()
    valor_diversidade_maxima, vertices_selecionados = calcular_diversidade_maxima_rapida(
        matriz_distancias, k, p=n
    )
    tempo_gasto = time.time() - start_time

    # Garantir conversão para inteiros nativos
    vertices_selecionados = [int(v) for v in vertices_selecionados]

    return valor_diversidade_maxima, 0.0, vertices_selecionados, tempo_gasto


def processar_pasta_instancias(pasta_instancias, output_csv):
    with open(output_csv, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Instância', 'Valor da Diversidade Máxima', 'GAP', 'Vértices Selecionados', 'Tempo Gasto (s)'])

        for arquivo in os.listdir(pasta_instancias):
            if arquivo.endswith(".txt"):
                caminho_arquivo = os.path.join(pasta_instancias, arquivo)
                try:
                    resultado = solver_diversidade(caminho_arquivo)
                    writer.writerow([arquivo, *resultado])
                except Exception as e:
                    print(f"Erro ao processar {arquivo}: {e}")


# Exemplo de uso
pasta_instancias = '../New Instances Carlos/uniformes inteiras'
output_csv = '../New Instances Carlos/uniformes inteiras/great_values_uniforme.csv'

processar_pasta_instancias(pasta_instancias, output_csv)
